<a href="https://colab.research.google.com/github/hammer1234567/Chatbot-Movie_Corpus/blob/master/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/My Drive/Kaggle/Chatbot')
!ls

 Chatbot.ipynb	'cornell movie-dialogs corpus'	 Cornell_movie_quotes_corpus


In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math

In [4]:
corpus_name = "cornell movie-dialogs corpus"
corpus = os.path.join("", corpus_name)

def print_line(file, n = 10):
  with open(file, 'rb') as datafile:
    lines = datafile.readlines()
  for line in lines[:n]:
    print(line)

print_line(os.path.join(corpus, "movie_lines.txt"))

b'L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!\n'
b'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!\n'
b'L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.\n'
b'L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?\n'
b"L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go.\n"
b'L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow\n'
b"L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie.\n"
b'L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No\n'
b'L870 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?\n'
b'L869 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Like my fear of wearing pastels?\n'


In [0]:
def loadlines(fileName, fields):
  lines = {}
  with open(fileName, 'r', encoding='iso-8859-1') as f:
    for line in f:
      values = line.split("+++$+++")
      lineObj = {}

      for i, field in enumerate(fields):
        lineObj[field] = values[i]

      lines[lineObj['lineID']] = lineObj
  return lines

In [0]:
def loadConversations(fileName, lines, fields):
  conversations = []
  with open(fileName, 'r', encoding='iso-8859-1') as f:
    for line in f:
      values = line.split("+++$+++")

      convObj = {}
      for i, field in enumerate(fields):
        convObj[field] = values[i]
      
      utterance_id_pattern = re.compile('L[0-9]+')
      lineIds = utterance_id_pattern.findall(convObj["utteranceIDs"])

      convObj["lines"] = []

      for lineId in lineIds:
        convObj["lines"].append(lines[lineId+(" ")])
      conversations.append(convObj)
  return conversations

In [0]:
def extractSentencePairs(conversations):
  qa_pairs = []
  for conversation in conversations:
    # Iterate over all lines of the conversations
    for i in range(len(conversation['lines']) - 1):
      inputLine = conversation["lines"][i]["text"].strip()
      targetLine = conversation["lines"][i+1]["text"].strip()

      if inputLine and targetLine:
        qa_pairs.append([inputLine, targetLine])
  return qa_pairs

In [8]:
datafile = os.path.join(corpus, "formatted_movie_lines.txt")

delimiter = '\t'

delimiter = str(codecs.decode(delimiter, "unicode_escape"))

# Initializing lines dictionaries and conversation list

lines = {}
conversations = []

MOVIE_LINES_FIELDS = ["lineID", "characterID", "movieID", "character", "text"]
MOVIE_CONVERSATIONS_FIELDS = ["character1ID", "character2ID", "movieID", "utteranceIDs"]


# Load lines and process conversations
print("\nProcessing corpus...")
lines = loadlines(os.path.join(corpus, "movie_lines.txt"), MOVIE_LINES_FIELDS)
print("\nLoading conversations...")


Processing corpus...

Loading conversations...


In [0]:
lines

{'L1045 ': {'character': ' BIANCA ',
  'characterID': ' u0 ',
  'lineID': 'L1045 ',
  'movieID': ' m0 ',
  'text': ' They do not!\n'},
 'L1044 ': {'character': ' CAMERON ',
  'characterID': ' u2 ',
  'lineID': 'L1044 ',
  'movieID': ' m0 ',
  'text': ' They do to!\n'},
 'L985 ': {'character': ' BIANCA ',
  'characterID': ' u0 ',
  'lineID': 'L985 ',
  'movieID': ' m0 ',
  'text': ' I hope so.\n'},
 'L984 ': {'character': ' CAMERON ',
  'characterID': ' u2 ',
  'lineID': 'L984 ',
  'movieID': ' m0 ',
  'text': ' She okay?\n'},
 'L925 ': {'character': ' BIANCA ',
  'characterID': ' u0 ',
  'lineID': 'L925 ',
  'movieID': ' m0 ',
  'text': " Let's go.\n"},
 'L924 ': {'character': ' CAMERON ',
  'characterID': ' u2 ',
  'lineID': 'L924 ',
  'movieID': ' m0 ',
  'text': ' Wow\n'},
 'L872 ': {'character': ' BIANCA ',
  'characterID': ' u0 ',
  'lineID': 'L872 ',
  'movieID': ' m0 ',
  'text': " Okay -- you're gonna need to learn how to lie.\n"},
 'L871 ': {'character': ' CAMERON ',
  'charac

In [0]:
conversations = loadConversations(os.path.join(corpus, "movie_conversations.txt"),lines, MOVIE_CONVERSATIONS_FIELDS)

In [10]:
print("\nWriting newly formatted file...")
with open(datafile, 'w', encoding='utf-8') as outputfile:
    writer = csv.writer(outputfile, delimiter=delimiter, lineterminator='\n')
    for pair in extractSentencePairs(conversations):
        writer.writerow(pair)

# Print a sample of lines
print("\nSample lines from file:")


Writing newly formatted file...

Sample lines from file:


Loading and Trimming The Data and making Vocabulary of Data

In [0]:
# Default word tokens
PAD_token = 0
SOS_token = 1
EOS_token = 2

class Voc:
  def __init__(self, name):
    self.name = name
    self.trimmed = False
    self.word2index = {}
    self.word2count = {}
    self.index2word = {PAD_token : "PAD", SOS_token : "SOS", EOS_token : "EOS"}
    self.num_words = 3

  def addSentence(self, sentence):
    for word in sentence.split(' '):
      self.addWord(word)

  def addWord(self, word):
    if word not in self.word2index:
      self.word2index[word] = self.num_words
      self.word2count[word] = 1
      self.index2word[self.num_words] = word
      self.num_words += 1
    else:
      self.word2count[word] += 1

  def trim(self, min_count):
    if self.trimmed:
      return
    self.trimmed = True

    keep_words = []

    for k, v in self.word2count.items():
      if v >= min_count:
        keep_words.append(k)
    print('keep_words {} / {} = {:.4f}'.format(
            len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
        ))
    
    self.word2index = {}
    self.word2count = {}
    self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
    self.num_words = 3 # Count default tokens

    for word in keep_words:
      self.addWord(word)


In [0]:
MAX_LENGTH = 10

def unicodeToAscii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

# lowercasing, trim, removing non-letter characters

def normalizeString(s):
  s = unicodeToAscii(s.lower().strip())
  s = re.sub(r"([.!?])", r" \1", s)
  s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
  s = re.sub(r"\s+", r" ", s).strip()

  return s

def readVocs(datafile, corpus_name):
  lines = open(datafile, encoding='utf-8').read().split('\n')
  # splitting every line into pairs and normalizing
  pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
  voc = Voc(corpus_name)

  return voc, pairs

In [20]:
def filterPair(p):
  # Input sequences need to preserve the last word for EOS token
  try:
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH
  except:
    return False

def filterPairs(pairs):
  return [pair for pair in pairs if filterPair(pair)]

def loadPrepareData(corpus, corpus_name, datafile, save_dir):
  voc, pairs = readVocs(datafile, corpus_name)
  pairs = filterPairs(pairs)

  for pair in pairs:
    voc.addSentence(pair[0])
    voc.addSentence(pair[1])
  
  return voc, pairs

# Load Data/Assemble pairs

save_dir = os.path.join("data", "save")
voc, pairs = loadPrepareData(corpus, corpus_name, datafile, save_dir)
# Print some pairs to validate
print("\npairs:")
for pair in pairs[:10]:
  print(pair)


pairs:
['there .', 'where ?']
['you have my word . as a gentleman', 'you re sweet .']
['hi .', 'looks like things worked out tonight huh ?']
['you know chastity ?', 'i believe we share an art instructor']
['have fun tonight ?', 'tons']
['well no . . .', 'then that s all you had to say .']
['then that s all you had to say .', 'but']
['but', 'you always been this selfish ?']
['do you listen to this crap ?', 'what crap ?']
['what good stuff ?', 'the real you .']


In [21]:
# trimming low count words
MIN_COUNT = 3

def trimRareWords(voc, pairs, MIN_COUNT):
  voc.trim(MIN_COUNT)

  keep_pairs = []

  for pair in pairs:
    input_sentence = pair[0]
    output_sentence =pair[1]

    keep_input = True
    keep_output = True

    for word in input_sentence.split(' '):
      if word not in voc.word2index:
        keep_input = False
        break

    for word in output_sentence.split(' '):
      if word not in voc.word2index:
        keep_output = False
        break
    # Only keep pairs that do not contain trimmed word(s) in their input or output sentence

    if keep_input and keep_output :
      keep_pairs.append(pair)

    return keep_pairs
    

pairs = trimRareWords(voc, pairs, MIN_COUNT)

keep_words 7823 / 18005 = 0.4345


In [0]:
def indexesFromSentence(voc, sentence):
  return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]

def zeroPadding(l, fillvalue = PAD_token):
  return list(itertools.zip_longest(*l, fillvalue=fillvalue))

def binaryMatrix(l, value = PAD_token):
  m = []
  for i, seq in enumerate(l):
    m.append([])
    for token in seq:
      if token == PAD_token:
        m[i].append(0)
      else:
        m[i].append(1)
  
  return m

# Returns padded input sequences tensor and lengths
def inputVar(l, voc):
  indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
  lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
  padList = zeroPadding(indexes_batch)
  padVar = torch.LongTensor(padList)

  return padVar, lengths

# Returns padded target sequence tensor, padding mask, and max target length
def outputVar(l, voc):
  indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
  max_target_len = max([len(indexes) for indexes in indexes_batch])
  padList = zeroPadding(indexes_batch)
  mask = binaryMatrix(padList)
  mask = torch.BoolTensor(mask)
  padVar = torch.LongTensor(padList)
  return padVar, mask, max_target_len

In [23]:
def batch2TrainData(voc, pair_batch):
  pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
  input_batch = []
  output_batch = []
  for pair in pair_batch:
    input_batch.append(pair[0])
    output_batch.append(pair[1])
  inp, lengths = inputVar(input_batch, voc)
  output, mask, max_target_len = outputVar(output_batch, voc)
  return inp, lengths, output, mask, max_target_len

small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print("input_variable:", input_variable)
print("lengths:", lengths)
print("target_variable:", target_variable)
print("mask:", mask)
print("max_target_len:", max_target_len)

input_variable: tensor([[3, 3, 3, 3, 3],
        [4, 4, 4, 4, 4],
        [2, 2, 2, 2, 2]])
lengths: tensor([3, 3, 3, 3, 3])
target_variable: tensor([[5, 5, 5, 5, 5],
        [6, 6, 6, 6, 6],
        [2, 2, 2, 2, 2]])
mask: tensor([[True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True]])
max_target_len: 3


In [0]:
class EncoderRNN(nn.Module):
  def __init__(self, hidden_size, embedding, n_layers = 1, dropout = 0):
    super(EncoderRNN, self).__init__()
    self.n_layers = n_layers
    self.hidden_size = hidden_size
    self.embedding = embedding

    self.gru = nn.GRU(hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout), bidirectional=True)

  def forward(self, input_seq, input_lengths, hidden = None):
    # convert word index to embeddings
    embedded = self.embedding(input_Seq)
    # Pack padded batch of sequences for RNN
    packed = nn.utils.rnn.pack_padded_sequence(embedded, input_lenghts)
    # Forward pass through GRU
    outputs, hidden = self.gru(packed, hidden)
    # Unpack padding
    outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)
    # Sum bidirectional GRU outputs
    outputs = outputs[:, :, :self.hidden_size] + outputs[:, :, self.hidden_size:]

    return outputs, hidden

In [30]:
class Attn(nn.Module):
  
  def __init__(self, method, hidden_size):
    super(Attn, self).__init__()
    self.method = method
    if self.method not in ['dot', 'general', 'concat']:
      raise ValueError(self.method, "is not an appropriate attention method.")
    self.hidden_size = hidden_size
    if self.method == 'general':
      self.attn = nn.Linear(self.hidden_size, hidden_size)
    elif self.method == 'concat':
      self.attn = nn.Linear(self.hidden_size *2 , hidden_size)
      self.v = nn.Parameter(torch.FloatTensor(hidden_size)
    
    return 0
    
  def dot_score(self, hidden, encoder_output):
    return torch.sum(hidden * encoder_output, dim = 2)

  def general_score(self, hidden, encoder_output):
    energy = self.attn(encoder_output)
    return torch.sum(hidden * eneregy, dim = 2)

  def concat_score(self, hidden, encoder_output):
    energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
    return torch.sum(self.v * energy, dim=2)

  def forward(self, hidden, encoder_outputs):
    if self.method == 'general':
      attn_energies = self.general_score(hidden, encoder_outputs)
    elif self.method == 'concat':
      attn_energies = self.concat_score(hidden, encoder_outputs)
    elif self.method == 'dot':
      attn_energies = self.dot_score(hidden, encoder_outputs)

    # Transpose max_length and batch_size dimensions
    attn_energies = attn_energies.t()

    # Return the softmax normalized probability scores (with added dimension)
    return F.softmax(attn_energies, dim=1).unsqueeze(1)


SyntaxError: ignored